## Mapping data from multiple json file 

- obtain genres name from genres json file 
- map the genres name to discover (movie) json file (from genre id to genres name)

In [1]:
import numpy as np 
import pandas as pd 
import json
import requests

In [2]:
api_key = "api_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [3]:
basic_url = "https://api.themoviedb.org/3/discover/movie?{}&include_adult=false&include_video=false&page={}"

In [4]:
requests.get(basic_url.format(api_key, 1)).json()

{'total_pages': 500,
 'page': 1,
 'total_results': 10000,
 'results': [{'adult': False,
   'backdrop_path': '/fTDzKoQIh1HeyjfpG5AHMi2jxAJ.jpg',
   'genre_ids': [28, 35],
   'original_language': 'en',
   'original_title': 'Chick Fight',
   'poster_path': '/4ZocdxnOO6q2UbdKye2wgofLFhB.jpg',
   'title': 'Chick Fight',
   'video': False,
   'vote_average': 6.1,
   'popularity': 2082.744,
   'overview': 'When Anna Wyncomb is introduced to an an underground, all-female fight club in order to turn the mess of her life around, she discovers she is much more personally connected to the history of the club than she could ever imagine.',
   'release_date': '2020-11-13',
   'id': 682377,
   'vote_count': 33},
  {'id': 724989,
   'backdrop_path': '/86L8wqGMDbwURPni2t7FQ0nDjsH.jpg',
   'genre_ids': [28, 53],
   'original_language': 'en',
   'original_title': 'Hard Kill',
   'poster_path': '/ugZW8ocsrfgI95pnQ7wrmKDxIe.jpg',
   'title': 'Hard Kill',
   'video': False,
   'vote_average': 5,
   'popular

In [5]:
json_list=[]

for i in range(1,501): #total_pages = 500
    r = requests.get(basic_url.format(api_key, i))
    if r.status_code != 200:
        continue 
    else:
        data = r.json()
        json_list = json_list + data['results']

In [6]:
df_json= pd.json_normalize(data=json_list)

In [7]:
df_json.set_index('id', inplace=True)
df_json.head()

,adult,backdrop_path,genre_ids,original_language,original_title,poster_path,title,video,vote_average,popularity,overview,release_date,vote_count
id,,,,,,,,,,,,,
682377,False,/fTDzKoQIh1HeyjfpG5AHMi2jxAJ.jpg,"[28, 35]",en,Chick Fight,/4ZocdxnOO6q2UbdKye2wgofLFhB.jpg,Chick Fight,False,6.1,2082.744,When Anna Wyncomb is introduced to an an under...,2020-11-13,33
724989,False,/86L8wqGMDbwURPni2t7FQ0nDjsH.jpg,"[28, 53]",en,Hard Kill,/ugZW8ocsrfgI95pnQ7wrmKDxIe.jpg,Hard Kill,False,5.0,1886.945,The work of billionaire tech CEO Donovan Chalm...,2020-10-23,201
650747,False,/lM9f4CnWU4RIY8HiVrwmrNKJpYN.jpg,[35],es,Historias lamentables,/sp4zXS3x4wHyL8wm8zLioiBrxuR.jpg,Unfortunate Stories,False,7.2,1342.233,"Four interconnected plots starring Ramón, a yo...",2020-11-19,35
741067,False,/aO5ILS7qnqtFIprbJ40zla0jhpu.jpg,"[28, 53, 12, 18]",en,Welcome to Sudden Death,/elZ6JCzSEvFOq4gNjNeZsnRFsvj.jpg,Welcome to Sudden Death,False,6.4,1199.820,Jesse Freeman is a former special forces offic...,2020-09-29,178
671039,False,/gnf4Cb2rms69QbCnGFJyqwBWsxv.jpg,"[53, 28, 18, 80]",fr,Bronx,/9HT9982bzgN5on1sLRmc1GMn6ZC.jpg,Rogue City,False,5.9,1174.741,Caught in the crosshairs of police corruption ...,2020-10-30,253


In [8]:
df_genres = df_json[['genre_ids']]
df_genres

,genre_ids
id,
682377,"[28, 35]"
724989,"[28, 53]"
650747,[35]
741067,"[28, 53, 12, 18]"
671039,"[53, 28, 18, 80]"
...,...
490263,[99]
363476,"[10749, 10770]"
295592,[99]


In [9]:
json_list = requests.get("https://api.themoviedb.org/3/genre/movie/list?api_key=2150281ee836fc2bb45a4164cf0de15c").json()['genres']

In [10]:
json_df = pd.DataFrame(json_list)
json_df.rename(columns={'id':'genre_id', 'name':'genre_name'}, inplace=True)
json_df.set_index('genre_id', inplace=True)

In [11]:
#flatten df_genres
df_genres_list = pd.DataFrame(df_genres['genre_ids'].to_list(), index=df_genres.index)
df_genres_list.head()

,0,1,2,3,4,5,6,7
id,,,,,,,,
682377,28.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN
724989,28.0,53.0,NaN,NaN,NaN,NaN,NaN,NaN
650747,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741067,28.0,53.0,12.0,18.0,NaN,NaN,NaN,NaN
671039,53.0,28.0,18.0,80.0,NaN,NaN,NaN,NaN


In [12]:
#map - genre_id to genre_name

for i in range(len(df_genres_list.columns)):
    df_genres_list[i] = df_genres_list[i].map(json_df.genre_name)

In [13]:
df_genres_list.head()

,0,1,2,3,4,5,6,7
id,,,,,,,,
682377,Action,Comedy,NaN,NaN,NaN,NaN,NaN,NaN
724989,Action,Thriller,NaN,NaN,NaN,NaN,NaN,NaN
650747,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741067,Action,Thriller,Adventure,Drama,NaN,NaN,NaN,NaN
671039,Thriller,Action,Drama,Crime,NaN,NaN,NaN,NaN


In [14]:
#back to list format

df_genres_list['genres_combined'] = df_genres_list[df_genres_list.columns[:]].apply(lambda x: '|'.join(x.dropna()), axis=1)

In [15]:
df_genres_list.head()

,0,1,2,3,4,5,6,7,genres_combined
id,,,,,,,,,
682377,Action,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,Action|Comedy
724989,Action,Thriller,NaN,NaN,NaN,NaN,NaN,NaN,Action|Thriller
650747,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comedy
741067,Action,Thriller,Adventure,Drama,NaN,NaN,NaN,NaN,Action|Thriller|Adventure|Drama
671039,Thriller,Action,Drama,Crime,NaN,NaN,NaN,NaN,Thriller|Action|Drama|Crime


In [16]:
#add genres_combined to df_json

df = df_json.merge(df_genres_list['genres_combined'], left_index=True, right_index=True)

In [17]:
df.head()

,adult,backdrop_path,genre_ids,original_language,original_title,poster_path,title,video,vote_average,popularity,overview,release_date,vote_count,genres_combined
id,,,,,,,,,,,,,,
682377,False,/fTDzKoQIh1HeyjfpG5AHMi2jxAJ.jpg,"[28, 35]",en,Chick Fight,/4ZocdxnOO6q2UbdKye2wgofLFhB.jpg,Chick Fight,False,6.1,2082.744,When Anna Wyncomb is introduced to an an under...,2020-11-13,33,Action|Comedy
724989,False,/86L8wqGMDbwURPni2t7FQ0nDjsH.jpg,"[28, 53]",en,Hard Kill,/ugZW8ocsrfgI95pnQ7wrmKDxIe.jpg,Hard Kill,False,5.0,1886.945,The work of billionaire tech CEO Donovan Chalm...,2020-10-23,201,Action|Thriller
650747,False,/lM9f4CnWU4RIY8HiVrwmrNKJpYN.jpg,[35],es,Historias lamentables,/sp4zXS3x4wHyL8wm8zLioiBrxuR.jpg,Unfortunate Stories,False,7.2,1342.233,"Four interconnected plots starring Ramón, a yo...",2020-11-19,35,Comedy
741067,False,/aO5ILS7qnqtFIprbJ40zla0jhpu.jpg,"[28, 53, 12, 18]",en,Welcome to Sudden Death,/elZ6JCzSEvFOq4gNjNeZsnRFsvj.jpg,Welcome to Sudden Death,False,6.4,1199.820,Jesse Freeman is a former special forces offic...,2020-09-29,178,Action|Thriller|Adventure|Drama
671039,False,/gnf4Cb2rms69QbCnGFJyqwBWsxv.jpg,"[53, 28, 18, 80]",fr,Bronx,/9HT9982bzgN5on1sLRmc1GMn6ZC.jpg,Rogue City,False,5.9,1174.741,Caught in the crosshairs of police corruption ...,2020-10-30,253,Thriller|Action|Drama|Crime


In [18]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 682377 to 553141
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              10000 non-null  bool   
 1   backdrop_path      9549 non-null   object 
 2   genre_ids          10000 non-null  object 
 3   original_language  10000 non-null  object 
 4   original_title     10000 non-null  object 
 5   poster_path        9920 non-null   object 
 6   title              10000 non-null  object 
 7   video              10000 non-null  bool   
 8   vote_average       10000 non-null  float64
 9   popularity         10000 non-null  float64
 10  overview           10000 non-null  object 
 11  release_date       9982 non-null   object 
 12  vote_count         10000 non-null  int64  
 13  genres_combined    10000 non-null  object 
dtypes: bool(2), float64(2), int64(1), object(9)
memory usage: 1.3+ MB


- genre_ids 10000 non-null 
- genres_combined 10000 non-null